<a href="https://colab.research.google.com/github/choki0715/lecture/blob/master/quant_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# accurate_size_comparison.py

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import gc
import psutil
import os

def get_model_size_mb(model):
    """모델의 실제 메모리 크기 계산"""
    param_size = 0
    buffer_size = 0

    for param in model.parameters():
        # BitsAndBytes 양자화된 경우 실제 비트 수 고려
        if hasattr(param, 'quant_state'):
            # 4-bit or 8-bit 양자화된 파라미터
            bits = 4 if '4bit' in str(param.dtype) else 8
            param_size += param.numel() * bits / 8  # bits to bytes
        else:
            param_size += param.numel() * param.element_size()

    for buffer in model.buffers():
        buffer_size += buffer.numel() * buffer.element_size()

    return (param_size + buffer_size) / (1024 * 1024)

def measure_gpu_memory():
    """GPU 메모리 사용량 측정"""
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        return torch.cuda.memory_allocated() / (1024 * 1024)
    return 0

def compare_quantization():
    """양자화 전후 크기 비교"""
    print("🔬 정확한 모델 크기 비교\n")

    model_name = "gpt2"

    # 1. 원본 모델
    print("="*50)
    print("1. 원본 모델 (FP32)")
    print("="*50)

    # GPU 메모리 초기화
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

    model_fp32 = AutoModelForCausalLM.from_pretrained(model_name)

    # CPU 메모리 측정
    fp32_params = sum(p.numel() for p in model_fp32.parameters())
    fp32_size = sum(p.numel() * p.element_size() for p in model_fp32.parameters()) / (1024**2)

    print(f"📊 파라미터 수: {fp32_params:,}")
    print(f"📊 이론적 크기: {fp32_size:.2f} MB")

    # 실제 메모리 사용량
    process = psutil.Process(os.getpid())
    fp32_memory = process.memory_info().rss / (1024 * 1024)
    print(f"📊 실제 RAM 사용: {fp32_memory:.2f} MB")

    del model_fp32
    gc.collect()
    torch.cuda.empty_cache()

    # 2. 8-bit 모델
    print("\n" + "="*50)
    print("2. 8-bit 양자화 모델")
    print("="*50)

    model_8bit = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_8bit=True,
        device_map="auto"
    )

    # 이론적 크기 (8-bit)
    expected_8bit = fp32_size / 4  # 32bit -> 8bit
    print(f"📊 예상 크기: {expected_8bit:.2f} MB")

    # GPU 메모리 측정
    if torch.cuda.is_available():
        gpu_8bit = torch.cuda.memory_allocated() / (1024**2)
        print(f"🎮 GPU 메모리: {gpu_8bit:.2f} MB")
        print(f"📉 GPU 감소율: {(1 - gpu_8bit/fp32_size)*100:.1f}%")

    # RAM 측정
    process = psutil.Process(os.getpid())
    ram_8bit = process.memory_info().rss / (1024 * 1024)
    print(f"📊 RAM 사용: {ram_8bit:.2f} MB")

    del model_8bit
    gc.collect()
    torch.cuda.empty_cache()

    # 3. 4-bit 모델
    print("\n" + "="*50)
    print("3. 4-bit 양자화 모델")
    print("="*50)

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    model_4bit = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto"
    )

    # 이론적 크기 (4-bit)
    expected_4bit = fp32_size / 8  # 32bit -> 4bit
    print(f"📊 예상 크기: {expected_4bit:.2f} MB")

    # GPU 메모리 측정
    if torch.cuda.is_available():
        gpu_4bit = torch.cuda.memory_allocated() / (1024**2)
        print(f"🎮 GPU 메모리: {gpu_4bit:.2f} MB")
        print(f"📉 GPU 감소율: {(1 - gpu_4bit/fp32_size)*100:.1f}%")

    # RAM 측정
    process = psutil.Process(os.getpid())
    ram_4bit = process.memory_info().rss / (1024 * 1024)
    print(f"📊 RAM 사용: {ram_4bit:.2f} MB")

    del model_4bit
    gc.collect()
    torch.cuda.empty_cache()

    # 4. 요약
    print("\n" + "="*50)
    print("📊 요약")
    print("="*50)
    print(f"FP32: {fp32_size:.2f} MB (기준)")
    print(f"INT8: ~{fp32_size/4:.2f} MB (이론적 75% 감소)")
    print(f"INT4: ~{fp32_size/8:.2f} MB (이론적 87.5% 감소)")

if __name__ == "__main__":
    compare_quantization()